In [4]:
import fitz
from df2gspread import df2gspread as d2g
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pandas.io.json import json_normalize
import sys
import re
test=True

In [5]:
%ls

Annotate Code.ipynb             Jupyter_and_Google_Sheets.json
Draft3-Full-annotated.pdf       README.md
Draft3-Full.pdf                 all_annotation.html
Draft3-Short.pdf                all_annotation.md
Example-Annotation.pdf          toc.csv


In [6]:
### READ IN PDF
annotated_doc = fitz.open("Draft3-Full-annotated.pdf")
original_doc = fitz.open("Draft3-Full.pdf")

In [7]:
class Annotation:
    def __init__(self, page_number, index, text, comment, code, areas, url):
        self.page_number = page_number
        self.index = index
        self.text = text
        self.comment = comment
        self.code = code
        self.areas = areas
        self.url = url
    
    def __repr__(self):
        return "{} - {}".format(self.page_number, self.index)
    
    def add_to_pdf(self, doc):
        """
        Adds the given annotation to the given PDF doc
        """
        page = doc[annotation.page_number-1]
        text_instances = page.searchFor(self.text)

        ## Add the highlight
        print text_instances
        for inst in text_instances:
            highlight = page.addHighlightAnnot(inst)

        footnote_loc = fitz.Rect(inst.x1,inst.y0-2,inst.x1+10,inst.y1-8)
        footnote = page.addFreetextAnnot(footnote_loc, str(self.index), fontsize=8)
        link = {'kind': 2, 
                'from': fitz.Rect(footnote_loc.x0-1, footnote_loc.y0-1, footnote_loc.x1+1, footnote_loc.y1+1),
                'type': 'uri',
                'uri': self.url}
        page.insertLink(link)
        comment = page.addTextAnnot(
            (10,footnote_loc.y0), 
            "{}. {}".format(self.index, self.comment), 
            icon="Comment")
if test:
    annotation = Annotation(page_number=233, index=1, text="Site Plan Dedication.", 
                              comment="Would this apply to site-plan lites?", 
                              code="Error", 
                              areas=["Missing Middle", "Parkland"],
                                url="https://htmlpreview.github.io/?https://github.com/kmclaugh/land-development-code-review/blob/master/all_annotation.html")
    print annotation
    annotation.add_to_pdf(annotated_doc)

233 - 1
[Rect(153.29989624023438, 388.762451171875, 252.2020263671875, 401.69970703125)]


In [8]:
annotated_doc.save(annotated_doc.name, incremental=True, encryption=0)

In [9]:
## Connect to Google Sheet
## Connect to our service account
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('./Jupyter_and_Google_Sheets.json', scope)
gc = gspread.authorize(credentials)
spreadsheet_key = '1IGMIjolRXo8zhPdFYkoma7BGTgZRekOZZnlFDKBY7Hs'

In [10]:
toc = original_doc.getToC(simple=False)
toc

[[1,
  'Draft3_ALDC_23-0_Cover_Web',
  1,
  {'kind': 1, 'page': 0, 'to': Point(0.0, -4.0), 'xref': 0, 'zoom': 0.0}],
 [2,
  'Chapter 23-1: Introduction',
  -1,
  {'file': '../ALDC_23-1 /ALDC_23-1_Introduction.pdf',
   'kind': 5,
   'page': 0,
   'to': Point(0.0, 0.0),
   'xref': 0,
   'zoom': 0.0}],
 [3,
  'Article 23-1A: General Provisions',
  -1,
  {'file': '../ALDC_23-1 /ALDC_23-1A_General_Provisions.pdf',
   'kind': 5,
   'page': 0,
   'to': Point(0.0, 0.0),
   'xref': 0,
   'zoom': 0.0}],
 [4,
  'Division 23-1A-1: Title, Purpose, and Scope',
  -1,
  {'file': '../ALDC_23-1 /ALDC_23-1A-1_title, purpose, and scope.pdf',
   'kind': 5,
   'page': 0,
   'to': Point(0.0, 0.0),
   'xref': 0,
   'zoom': 0.0}],
 [4,
  'Division 23-8D-2: Variances and Appeals',
  -1,
  {'file': '../ALDC_23-1 /ALDC_23-1A-1_title, purpose, and scope.pdf',
   'kind': 5,
   'page': 4,
   'to': Point(0.0, 0.0),
   'xref': 0,
   'zoom': 0.0}],
 [4,
  'Division 23-8D-2: Variances and Appeals',
  -1,
  {'file': '../

In [126]:
# chapter_regex = re.compile('_(23-\d+)_')
# article_regex = re.compile('(Article 23-\d+\w):')
# division_regex = re.compile('(Division 23-\d\w-\d):')
# section_regex = re.compile('(23-\d+\w-\d\d\d\d)\s')
all_regex = re.compile('23-(\d+)([A-Z])?-?(\d)?(\d+)?:?_?\s?\t?')

def title_to_item(title):
#     chapter_match = chapter_regex.search(title)
#     article_match = article_regex.search(title)
#     division_match = division_regex.search(title)
#     section_match = section_regex.search(title)
    all_match = all_regex.search(title)
    
    chapter = all_match.group(1)
    article = all_match.group(2)
    division = all_match.group(3)
    section = all_match.group(4)
    name = title[all_match.end():]
    
    if section:
        return {"level": "section", "level-number":4, "name":name, "chapter": int(chapter), "article": article, "division": division, "section": section}
    elif division:
        return {"level": "division", "level-number":3, "name":name, "chapter": int(chapter), "article": article, "division": division}
    elif article:
        return {"level": "article", "level-number":2, "name":name, "chapter": int(chapter), "article": article}
    elif chapter:
        return {"level": "chapter", "level-number":1, "name":name, "chapter": int(chapter)}
    
#     if chapter_match != None:
#         name = title[chapter_match.end():]
#         chapter = chapter_match.group(1)
#         return {"chapter": chapter, "name": name}
    
#     else if division_match != None:
#         group1 = chapter_match.group(1)
#         name = title[division_match.end():]
#         chapter = group1[10:14]
#         chapter = group1[10:14]
        
#         return {"chapter": chapter, "name": name}
    
#     else if article_match != None:
#         name = title[chapter_match.end():]
#         chapter = chapter_match.group(1)
#         return {"chapter": chapter, "name": name}
    
#     else if section_match != None:
#         name = title[chapter_match.end():]
#         chapter = chapter_match.group(1)
#         return {"chapter": chapter, "name": name}


In [131]:
def find_last_page(row):
    """
    Finds the last page of the given item
    """
    index = int(row.name)
    search_df = tocdf.loc[index:]
    search_df = search_df[search_df['level-number'] <= row['level-number']]
    if search_df.shape[0] > 1:
        return search_df.iloc[1].Page
    else:
        return original_doc.pageCount

#page_count(tocdf.iloc[0], 0, tocdf)

In [142]:
tocdf = pd.DataFrame.from_records(toc, columns=["Level", "Title", "Page", "Destination Dict"])
tocdf = pd.concat([tocdf, pd.DataFrame(tocdf["Destination Dict"].tolist())], axis=1)
tocdf = tocdf.drop(['Destination Dict'], axis=1)
tocdf = tocdf.drop(['file', "to", "xref", "zoom"], axis=1)
items = tocdf["Title"].apply(title_to_item)
tocdf = pd.concat([tocdf,  pd.DataFrame(items.tolist())], axis=1)
tocdf = tocdf[tocdf["kind"] != 5]

tocdf = tocdf.sort_values(by=['chapter', 'article', 'division', 'section', 'Page'], na_position='first')
tocdf = tocdf.drop_duplicates(subset=['chapter', 'article', 'division', 'section'], keep='last')
tocdf = tocdf.reset_index(drop=True)


tocdf["link"] = tocdf["Page"].apply(lambda x: "https://slideruletech.com/pdfs/Draft3-Full-annotated.pdf#page="+str(x))

tocdf["Last Page"] = tocdf.apply(find_last_page, axis=1)
tocdf["Page Count"] = tocdf["Last Page"] - tocdf["Page"]

tocdf = tocdf[[u'level', u'chapter', u'article', u'division', u'section', u'name', u'Page', u'Last Page', u'Page Count', u'link']]
tocdf.to_csv("toc.csv")
print(tocdf.shape)
tocdf

,level,chapter,article,division,section,name,Page,Last Page,Page Count,link
0,chapter,0,NaN,NaN,NaN,Cover_Web,1,13,12,https://slideruletech.com/pdfs/Draft3-Full-ann...
1,chapter,1,NaN,NaN,NaN,Introduction_Web,13,53,40,https://slideruletech.com/pdfs/Draft3-Full-ann...
2,article,1,A,NaN,NaN,General Provisions,17,35,18,https://slideruletech.com/pdfs/Draft3-Full-ann...
3,division,1,A,1,NaN,"Title, Purpose, and Scope",19,23,4,https://slideruletech.com/pdfs/Draft3-Full-ann...
4,section,1,A,1,010,Title and Citation,19,19,0,https://slideruletech.com/pdfs/Draft3-Full-ann...
5,section,1,A,1,020,Purpose,19,20,1,https://slideruletech.com/pdfs/Draft3-Full-ann...
6,section,1,A,1,030,Scope and Effect,20,23,3,https://slideruletech.com/pdfs/Draft3-Full-ann...
7,division,1,A,2,NaN,Authority,23,27,4,https://slideruletech.com/pdfs/Draft3-Full-ann...
8,section,1,A,2,010,Scope of Municipal Authority,23,23,0,https://slideruletech.com/pdfs/Draft3-Full-ann...
9,section,1,A,2,020,Implied Authority,23,24,1,https://slideruletech.com/pdfs/Draft3-Full-ann...


In [143]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

#tocdf["Title"] = tocdf["Title"].apply(is_not_ascii)
d2g.upload(tocdf, "15PK8zHAPuk8S-erSySqwxHlD9FyLkVBkkbemFwZLe5g", "table of contents breakdown", credentials=credentials, row_names=True)

reload(sys)
sys.setdefaultencoding("ASCII")

In [ ]:
tocdf.head()

In [136]:
original_doc.search("Chapter 23-1: Introduction")

AttributeError: 'Document' object has no attribute 'search'